In [1]:
# Pull the data from the Google Drive and Move it to the folder 'New Data'


import os
import shutil
import sqlite3
import xml.etree.ElementTree as ET
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
from docx import Document
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import shutil
import tkinter as tk
from tkinter import filedialog

weight = 225
def select_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    folder_path = filedialog.askdirectory(initialdir=r"H:\Pitching\Data", title="Select Folder")
    if folder_path:
        copy_and_move_files(folder_path)


def extract_name(folder_name):
    # Extract the full name from the folder name
    # Assuming the folder name is in the format "Last, First"
    parts = folder_name.split(", ")
    if len(parts) == 2:
        first_name = parts[1].strip()
        last_name = parts[0].strip()
        full_name = f"{first_name} {last_name}"
    else:
        full_name = folder_name.strip()
    return full_name


def copy_and_move_files(base_path):
    # Get the base folder name
    base_folder = os.path.basename(base_path)

    # Get list of folders in the base path
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    # Sort folders by creation time
    folders.sort(key=lambda x: os.path.getctime(os.path.join(base_path, x)), reverse=True)

    if not folders:
        print("No folders found in the specified directory.")
        return

    latest_folder = folders[0]
    folder_path = os.path.join(base_path, latest_folder)

    # Get creation time of the latest folder
    creation_time = os.path.getctime(folder_path)
    creation_date = datetime.datetime.fromtimestamp(creation_time).strftime('%m-%d-%y')

    # Find the files to copy and rename
    files_to_copy_rename = []
    for file in os.listdir(folder_path):
        if file.lower() == 'session.xml' or file.lower() == 'session_data.xml':
            files_to_copy_rename.append(file)

    if not files_to_copy_rename:
        print("No XML files named 'session.xml' or 'session_data.xml' found in the latest folder.")
        return

    # Construct new file names
    new_file_names = []
    initials = extract_name(base_folder)
    for file in files_to_copy_rename:
        if file.lower() == 'session.xml':
            new_file_name = f"session_{initials}_{creation_date}.xml"
        elif file.lower() == 'session_data.xml':
            new_file_name = f"session_data_{initials}_{creation_date}.xml"
        else:
            continue
        new_file_names.append(new_file_name)

    # Create temporary directory to store renamed copies
    temp_dir = os.path.join(base_path, "temp_rename")
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    # Copy and rename the files to the temporary directory
    for original_file, new_file_name in zip(files_to_copy_rename, new_file_names):
        original_file_path = os.path.join(folder_path, original_file)
        new_file_path = os.path.join(temp_dir, new_file_name)
        shutil.copyfile(original_file_path, new_file_path)
        print(f"File '{original_file}' copied and renamed to: {new_file_path}")

    # Copy the renamed files to the destination folder
    new_folder_path = r"G:\My Drive\Pitching Data\New Data"
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    for new_file_name in new_file_names:
        temp_file_path = os.path.join(temp_dir, new_file_name)
        destination_file_path = os.path.join(new_folder_path, new_file_name)
        shutil.copyfile(temp_file_path, destination_file_path)
        print(f"File '{new_file_name}' copied to: {destination_file_path}")

    # Clean up temporary directory
    shutil.rmtree(temp_dir)


# Prompt user to select the folder
select_folder()


# Source and destination directory paths
source_dir = r"G:\My Drive\Pitching Data\New Data"
destination_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"

# Iterate over files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith('.xml'):
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(destination_dir, file_name)
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file_name}' moved to: {destination_file_path}")

print("XML files copied successfully!")

# This code is *chef's kiss* perfect. Does everything for parsing for variables, iterates over it
# multiple times, and includes the specific filename for each trial. I love it.


# Connect to the SQLite database
conn = sqlite3.connect("grading_equation_new.db")
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute('DROP TABLE IF EXISTS variables')

# Create the variables table
cursor.execute('''
   CREATE TABLE variables (
       id INTEGER PRIMARY KEY AUTOINCREMENT,
       session_data_id TEXT NOT NULL,
       file_name TEXT NOT NULL,
       Pitch TEXT,
       Score REAL,
       Linear_Pelvis_Speed REAL,
       HSS_Footplant REAL,
       Pelvis_Ang_Footplant REAL,
       Trunk_Ang_Footplant REAL,
       Pelvic_Obl REAL,
       Front_Leg_Brace REAL,
       Front_Leg_Var_Val REAL,
       Lead_Leg_Midpoint REAL,
       Lead_Leg_GRF_y REAL,
       Lead_Leg_GRF_z REAL,
       Lead_Leg_GRF_x REAL,
       Horizontal_Abduction REAL,
       Shld_ER_Footplant REAL,
       Shld_ER_Max REAL,
       Lateral_Trunk_Tilt REAL,
       Pelvis_Ang_Velo REAL,
       Torso_Ang_Velo REAL,
       Arm_Ang_Velo REAL,
       MPH REAL
   )
''')

# Commit the changes before proceeding to parsing XML files
conn.commit()

# List of XML files to parse (replace this with your actual directory path)
directory_path = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"

while True:
    # Get a list of XML files in the specified directory
    xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]

    # Track the number of new data entries
    new_data_entries = 0

    for file_name in xml_files:
        try:
            # Check if the file has already been processed
            cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
            count = cursor.fetchone()[0]

            if count == 0:
                # Parse the XML file
                tree = ET.parse(file_name)
                root = tree.getroot()

                # Extract session_data_id from the file name
                session_data_id_match = re.search(r"session_data_(.+)\.xml", file_name)
                if session_data_id_match:
                    session_data_id = session_data_id_match.group(1)
                else:
                    session_data_id = None

                # Check if session_data_id is None, and skip the file if it is
                if session_data_id is not None:
                    # Iterate over all occurrences of "owner" element with a value containing "Fastball"
                    for owner_element in root.findall('.//owner'):
                        if 'Fastball' in owner_element.get('value', ''):
                            fastball_owner = owner_element
                            # Extract and insert data into the database
                            linear_pelvis_speed = None
                            hss_footplant = None
                            pelvis_ang_fp = None
                            trunk_ang_fp = None
                            pelvis_obl = None
                            front_leg_brace = None
                            front_leg_var_val = None
                            lead_leg_midpoint = None
                            lead_leg_grf_y = None
                            lead_leg_grf_z = None
                            lead_leg_grf_x = None
                            horizontal_abduction = None
                            shld_er_fp = None
                            shld_er_max = None
                            lateral_trunk_tilt = None
                            pelvis_ang_velo = None
                            torso_ang_velo = None
                            arm_ang_velo = None

                            # Extract the value of "owner"
                            pitch_value = fastball_owner.get('value', '')

                            for variable_element in fastball_owner.findall('.//name[@value]'):
                                variable_name = variable_element.attrib['value']
                                component_x_element = variable_element.find('./component[@value="X"]')
                                component_y_element = variable_element.find('./component[@value="Y"]')
                                component_z_element = variable_element.find('./component[@value="Z"]')

                                if variable_name == "MaxPelvisLinearVel_MPH" and component_y_element is not None:
                                    linear_pelvis_speed = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Hip Shoulders Sep@Footstrike" and component_z_element is not None:
                                    hss_footplant = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Angle@Footstrike" and component_z_element is not None:
                                    pelvis_ang_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Trunk_Angle@Footstrike" and component_z_element is not None:
                                    trunk_ang_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Knee_Angle@Footstrike" and component_x_element is not None:
                                    lead_knee_ang_at_footstrike = float(
                                        component_x_element.attrib['data'].replace(',', '.'))
                                    lead_knee_ang_at_release = float(root.find(
                                        './/name[@value="Lead_Knee_Angle@Release"]/component[@value="X"]').attrib[
                                                                         'data'].replace(',', '.'))
                                    front_leg_brace = lead_knee_ang_at_footstrike - lead_knee_ang_at_release
                                if variable_name == "Lead_Knee_Angle@Footstrike" and component_y_element is not None:
                                    lead_knee_var_val_fp = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Knee_Angle@Release" and component_y_element is not None:
                                    lead_knee_var_val_rel = float(component_y_element.attrib['data'].replace(',', '.'))
                                    # Calculate the difference and store in front_leg_var_val
                                    front_leg_var_val = lead_knee_var_val_fp - lead_knee_var_val_rel
                                if variable_name == "Pelvis_Angle@Footstrike" and component_y_element is not None:
                                    pelvis_obl_fp = float(component_y_element.attrib['data'].replace(',', '.'))
                                    pelvis_angle_release_element = root.find(
                                        './/name[@value="Pelvis_Angle@Release"]/component[@value="Y"]')
                                    if pelvis_angle_release_element is not None:
                                        pelvis_obl_release = float(
                                            pelvis_angle_release_element.attrib['data'].replace(',', '.'))
                                        pelvis_obl = pelvis_obl_release - pelvis_obl_fp
                                elif variable_name == "Lead_Leg_GRF_mag_Midpoint_FS_Release" and component_x_element is not None:
                                    lead_leg_midpoint = abs(float(component_x_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Lead_Leg_GRF_min" and component_y_element is not None:
                                    lead_leg_grf_y = abs(float(component_y_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Lead_Leg_GRF_max" and component_z_element is not None:
                                    lead_leg_grf_z = float(component_z_element.attrib['data'].replace(',', '.'))
                                if variable_name == "Lead_Leg_GRF_max" and component_x_element is not None:
                                    lat_force_max = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Leg_GRF_min" and component_x_element is not None:
                                    lat_force_min = float(component_x_element.attrib['data'].replace(',', '.'))
                                    # Calculate the sum of absolute values and store in lead_leg_grf_x
                                    lead_leg_grf_x = abs(lat_force_max) + abs(lat_force_min)
                                elif variable_name == "Pitching_Shoulder_Angle@Footstrike" and component_x_element is not None:
                                    horizontal_abduction = abs(
                                        float(component_x_element.attrib['data'].replace(',', '.')))
                                if variable_name == "Pitching_Shoulder_Angle@Footstrike":
                                    if component_z_element is not None:
                                        shld_er_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Shoulder_Angle_Max" and component_z_element is not None:
                                    shld_er_max = abs(float(component_z_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Trunk_wrt_Pelvis_FE@Release" and component_y_element is not None:
                                    lateral_trunk_tilt = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Ang_Vel_max" and component_x_element is not None:
                                    pelvis_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Thorax_Ang_Vel_max" and component_x_element is not None:
                                    torso_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Humerus_Ang_Vel_max" and component_x_element is not None:
                                    arm_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))

                                linear_pelvis_speed = linear_pelvis_speed if linear_pelvis_speed is not None else 4
                                hss_footplant = hss_footplant if hss_footplant is not None else 30
                                pelvis_ang_fp = pelvis_ang_fp if pelvis_ang_fp is not None else 60
                                trunk_ang_fp = trunk_ang_fp if trunk_ang_fp is not None else 90
                                pelvis_obl = pelvis_obl if pelvis_obl is not None else 2
                                front_leg_brace = front_leg_brace if front_leg_brace is not None else 8
                                front_leg_var_val = front_leg_var_val if front_leg_var_val is not None else 0
                                lead_leg_midpoint = lead_leg_midpoint if lead_leg_midpoint is not None else 1.5
                                lead_leg_grf_y = lead_leg_grf_y if lead_leg_grf_y is not None else .7
                                lead_leg_grf_z = lead_leg_grf_z if lead_leg_grf_z is not None else 1.5
                                lead_leg_grf_x = lead_leg_grf_x if lead_leg_grf_x is not None else .25
                                horizontal_abduction = horizontal_abduction if horizontal_abduction is not None else 20
                                shld_er_fp = shld_er_fp if shld_er_fp is not None else 45
                                lateral_trunk_tilt = lateral_trunk_tilt if lateral_trunk_tilt is not None else 35
                                shld_er_max = shld_er_max if shld_er_max is not None else 160
                                pelvis_ang_velo = pelvis_ang_velo if pelvis_ang_velo is not None else 600
                                torso_ang_velo = torso_ang_velo if torso_ang_velo is not None else 1000
                                arm_ang_velo = arm_ang_velo if arm_ang_velo is not None else 5000
                                    
                            lead_leg_grf_y = (lead_leg_grf_y / (weight * 4.44822))
                            lead_leg_grf_z = (lead_leg_grf_z / (weight * 4.44822))
                            lead_leg_grf_x = (lead_leg_grf_x / (weight * 4.44822))
                            lead_leg_midpoint = (lead_leg_midpoint / (weight * 4.44822))

                            print(lead_leg_grf_y, lead_leg_grf_z, lead_leg_grf_x, lead_leg_midpoint)

                                # Insert the data into the database
                            cursor.execute('''
                               INSERT INTO variables (
                                   session_data_id,
                                   file_name,
                                   Pitch,
                                   Linear_Pelvis_Speed,
                                   HSS_Footplant,
                                   Pelvis_Ang_Footplant,
                                   Trunk_Ang_Footplant,
                                   Pelvic_Obl,
                                   Front_Leg_Brace,
                                   Front_Leg_Var_Val,
                                   Lead_Leg_Midpoint,
                                   Lead_Leg_GRF_y,
                                   Lead_Leg_GRF_z,
                                   Lead_Leg_GRF_x,
                                   Horizontal_Abduction,
                                   Shld_ER_Footplant,
                                   Shld_ER_Max,
                                   Lateral_Trunk_Tilt,
                                   Pelvis_Ang_Velo,
                                   Torso_Ang_Velo,
                                   Arm_Ang_Velo,
                                   MPH
                               ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                           ''', (
                            session_data_id, file_name, pitch_value, linear_pelvis_speed, hss_footplant, pelvis_ang_fp,
                            trunk_ang_fp, pelvis_obl, front_leg_brace, front_leg_var_val, lead_leg_midpoint, lead_leg_grf_y,
                            lead_leg_grf_z, lead_leg_grf_x, horizontal_abduction, shld_er_fp, shld_er_max,
                            lateral_trunk_tilt, pelvis_ang_velo, torso_ang_velo, arm_ang_velo, None))

                            # Increment the count of new data entries
                            new_data_entries += 1
                else:
                    print(f"Skipping file {file_name} due to missing session_data_id.")

        except Exception as e:
            # Print the error and the file name where it occurred
            print(f"Error processing file {file_name}: {e}")

    # Commit the changes for each file
    conn.commit()

    # If no new data entries, break out of the loop
    if new_data_entries == 0:
        break

# Close the connection
conn.close()

print("Data inserted into the database.")

# Successfully adds MPH to the appropriate spot in the database


# Connect to the database
conn = sqlite3.connect("grading_equation_new.db")
cursor = conn.cursor()

directory_path = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\MPH Data\High School"

# Get a list of XML files in the specified directory
xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]

# Track the number of new data entries
new_data_entries = 0

for file_name in xml_files:
    try:
        # Check if the file has already been processed
        cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
        count = cursor.fetchone()[0]

        if count == 0:
            # Parse the XML file
            tree = ET.parse(file_name)
            root = tree.getroot()

            # Extract session_data_id from the file name
            session_data_id_match = re.search(r"session_(.+)\.xml", file_name)
            if session_data_id_match:
                session_data_id = session_data_id_match.group(1)
            else:
                session_data_id = None

            # Check if session_data_id is None, and skip the file if it is
            if session_data_id is not None:
                # Iterate over all occurrences of "Measurement" element
                for measurement_element in root.findall('.//Measurement'):
                    pitch_value = measurement_element.get('Filename')
                    used_element = measurement_element.find('./Fields/Used')

                    if pitch_value is not None and used_element is not None:
                        # Replace "qtm" with "c3d" in pitch_value
                        pitch_value = pitch_value.replace("qtm", "c3d")

                        # Print statement for debugging
                        print(f"Filename: {pitch_value}, Used: {used_element.text}")

                        if 'Fastball' in pitch_value and used_element.text == 'True':
                            # Extract the values of "Comments" and "Ball_speed"
                            comments_value_element = measurement_element.find('./Fields/Comments')

                            if comments_value_element is not None:
                                print(f"MPH: {comments_value_element.text}")
                                print(f"Session ID: {session_data_id}")

                                mph_value = float(comments_value_element.text)

                                # Update the MPH value in the "variables" table
                                cursor.execute('''
                                   UPDATE variables
                                   SET MPH = ?
                                   WHERE session_data_id = ? AND Pitch = ?
                               ''', (mph_value, session_data_id, pitch_value))

                                # Increment the new data entries counter
                                new_data_entries += 1
                    else:
                        print("Filename or Used element not found.")

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

# Commit the changes
conn.commit()

# Close the connection
conn.close()

print(f"Number of new data entries: {new_data_entries}")
print("Data inserted into the database.")

# Generates Score


# Connect to the SQLite database
conn = sqlite3.connect("grading_equation_new.db")
cursor = conn.cursor()

# Select all rows from the 'variables' table
cursor.execute('SELECT * FROM variables')
rows = cursor.fetchall()

# Iterate through each row and calculate the score
for row in rows:
    linear_pelvis_speed = row[5]
    hss_footplant = row[6]
    pelvis_ang_fp = row[7]
    trunk_ang_fp = row[8]
    pelvis_obl = row[9]
    front_leg_brace = row[10]
    front_leg_var_val = row[11]
    lead_leg_midpoint = row[12]
    lead_leg_grf_y = row[13]
    lead_leg_grf_z = row[14]
    lead_leg_grf_x = row[15]
    horizontal_abduction = row[16]
    shld_er_fp = row[17]
    shld_er_max = row[18]
    lateral_trunk_tilt = row[19]
    pelvis_ang_velo = row[20]
    torso_ang_velo = row[21]
    arm_ang_velo = row[22]
    MPH = row[23]

    # Check for None values and replace with 0
    linear_pelvis_speed = linear_pelvis_speed if linear_pelvis_speed is not None else 4
    hss_footplant = hss_footplant if hss_footplant is not None else 30
    pelvis_ang_fp = pelvis_ang_fp if pelvis_ang_fp is not None else 60
    trunk_ang_fp = trunk_ang_fp if trunk_ang_fp is not None else 90
    pelvis_obl = pelvis_obl if pelvis_obl is not None else 2
    front_leg_brace = front_leg_brace if front_leg_brace is not None else 8
    front_leg_var_val = front_leg_var_val if front_leg_var_val is not None else 0
    lead_leg_midpoint = lead_leg_midpoint if lead_leg_midpoint is not None else 1.5
    lead_leg_grf_y = lead_leg_grf_y if lead_leg_grf_y is not None else .7
    lead_leg_grf_z = lead_leg_grf_z if lead_leg_grf_z is not None else 1.5
    lead_leg_grf_x = lead_leg_grf_x if lead_leg_grf_x is not None else .25
    horizontal_abduction = horizontal_abduction if horizontal_abduction is not None else 20
    shld_er_fp = shld_er_fp if shld_er_fp is not None else 45
    lateral_trunk_tilt = lateral_trunk_tilt if lateral_trunk_tilt is not None else 35
    shld_er_max = shld_er_max if shld_er_max is not None else 160
    pelvis_ang_velo = pelvis_ang_velo if pelvis_ang_velo is not None else 600
    torso_ang_velo = torso_ang_velo if torso_ang_velo is not None else 1000
    arm_ang_velo = arm_ang_velo if arm_ang_velo is not None else 5000
    MPH = MPH if MPH is not None else 75

    # Calculate the score using the specified equation
    score = (0.05 * linear_pelvis_speed) + (0.15 * front_leg_brace) + \
            (5 * lead_leg_midpoint) + \
            (0.55 * horizontal_abduction) + (0.012 * torso_ang_velo) + (.05 * pelvis_obl) + (
                        .6 * trunk_ang_fp) - (.45 * pelvis_ang_fp) + (.1 * shld_er_max) + (.1 * front_leg_var_val) + (
                        .05 * pelvis_ang_velo) + MPH

    # Update the 'Score' column in the database
    cursor.execute('UPDATE variables SET Score = ? WHERE id = ?', (score, row[0]))

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Scores updated in the database.")


from datetime import date
import os
import sqlite3
import tempfile
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np

# Define the directory to save exported graphs and documents
export_dir = r"G:\My Drive\Percentile Reports\High School"
os.makedirs(export_dir, exist_ok=True)

# Connect to the SQLite databases
conn_new = sqlite3.connect("grading_equation_new.db")
conn_reference = sqlite3.connect("grading_equation_reference_data_HS.db")
cursor_new = conn_new.cursor()
cursor_reference = conn_reference.cursor()

try:
    cursor_new.execute('SELECT * FROM variables ORDER BY CAST(Score AS REAL) DESC LIMIT 1')
    row_with_highest_score = cursor_new.fetchone()
    highest_score_values = {col[0]: row_with_highest_score[idx] for idx, col in enumerate(cursor_new.description)}

    session_data_id = highest_score_values.get('session_data_id')  # Assuming 'session_data_id' is in the fetched data

    variables_to_include = ['Score', 'Linear_Pelvis_Speed', 'Pelvic_Obl', 'HSS_Footplant', 'Pelvis_Ang_Footplant', 'Trunk_Ang_Footplant', 'Front_Leg_Brace',
                            'Front_Leg_Var_Val', 'Lead_Leg_Midpoint', 'Horizontal_Abduction', 'Shld_ER_Footplant', 'Shld_ER_Max',
                            'Lateral_Trunk_Tilt', 'Pelvis_Ang_Velo', 'Torso_Ang_Velo', 'Arm_Ang_Velo', 'MPH']

    docx_filename = os.path.join(export_dir, f"8ctane Percentiles {session_data_id}.docx")
    doc = Document()
    doc.add_picture(os.path.join(export_dir, "8ctane - Faded 8 to Blue.png"), width=Inches(2.5))
    doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph("\nPlayer's Name: ")  # Modify as needed
    doc.add_paragraph(f"Date: {date.today().strftime('%B %d, %Y')}")

    # Specify the variables after which you want a page break
    variables_with_page_breaks = ['HSS_Footplant', 'Trunk_Ang_Footplant', 'Lead_Leg_Midpoint', 'Horizontal_Abduction', 'Lateral_Trunk_Tilt', 'Arm_Ang_Velo']

    with tempfile.TemporaryDirectory() as tmpdirname:
        for i, column_name in enumerate(variables_to_include):
            cursor_reference.execute(f'SELECT CAST({column_name} AS REAL) FROM variables WHERE {column_name} IS NOT NULL')
            reference_values = [value[0] for value in cursor_reference.fetchall() if value[0] is not None]

            current_value = highest_score_values.get(column_name)
            if reference_values and current_value is not None:
                percentile = stats.percentileofscore(reference_values, current_value, kind='mean')

                # Generate histograms
                plt.figure(facecolor='#181818')
                ax = plt.subplot(111, facecolor='#303030')
                n, bins, patches = ax.hist(reference_values, bins=20, color='cornflowerblue', alpha=0.7, edgecolor='white', label='Reference Data')
                ax.axvline(x=current_value, color='red', linestyle='dashed', linewidth=2, label='Individual Score')
                
                # Set axis ticks
                xticks = np.linspace(min(reference_values), max(reference_values), num=10)  # Adjust 'num' for more or fewer ticks
                ax.set_xticks(xticks)  # Set x-axis ticks
                ax.set_yticks(range(0, int(max(ax.get_yticks())) + 1, 5))  # Set y-axis ticks, adjusting step as needed
                
                # Conditional formatting for x-axis labels
                if column_name in ['Lead_Leg_Midpoint', 'Linear_Pelvis_Speed']:
                    ax.set_xticklabels([f'{x:.2f}' for x in xticks], color='slategrey')  # Two decimal places
                else:
                    ax.set_xticklabels([f'{int(round(x))}' for x in xticks], color='slategrey')  # Nearest whole number
                
                # Add labels to y-axis ticks, rounded to the nearest whole number
                ax.set_yticklabels([f'{int(round(y))}' for y in ax.get_yticks()], color='slategrey')
                
                ax.set_xlabel(column_name, color='slategrey')
                ax.set_ylabel('Frequency', color='slategrey')
                ax.set_title(f'Histogram of {column_name}', color='lightgrey')
                ax.grid(color='dimgrey')
                ax.legend(facecolor='black', edgecolor='grey', prop={'size': 'small'}, labelcolor='grey')
                plt.text(0.95, 0.05, f'Percentile: {round(percentile, 2)}%\nValue: {round(current_value, 2)}', 
                         ha='right', va='bottom', transform=ax.transAxes, color='white', fontsize=10, backgroundcolor='#181818')
                
                histogram_filename = os.path.join(tmpdirname, f'{column_name}.png')
                plt.savefig(histogram_filename, bbox_inches='tight', facecolor='#181818')
                plt.close()

                # Add descriptions and titles based on column name
                if column_name == "Score":
                    doc.add_paragraph("8ctane Score", style='Title')
                    doc.add_paragraph(
                        "The 8ctane score breaks down your pitch and summarizes it in a single score. This score is a product of "
                        "over twenty variables that are analyzed and interpreted to determine how efficient your pitching mechanics"
                        " are."
                        "\n", style='Heading 2')
                elif column_name == "Linear_Pelvis_Speed":
                    doc.add_paragraph("Pelvis", style='Title')
                    doc.add_paragraph("Maintaining a stable and neutral pelvis throughout the pitching delivery is crucial in a "
                                      "foundational sound pitch delivery."
                                      "", style='Heading 2')
                elif column_name == "HSS_Footplant":
                    doc.add_paragraph("Hip Shoulder Separation at FootPlant", style='Title')
                    doc.add_paragraph("Hip shoulder separation is achieved by clearing the pelvis towards home while allowing the "
                                      "trunk to stay as closed as possible. With pelvis angle at footplant and total hip shoulder "
                                      "separation being linked to increased ball velocity, these variables are instrumental in creating "
                                      "and maintaining ball velocity."
                                      "", style='Heading 2')
                elif column_name == "Front_Leg_Brace":
                    doc.add_paragraph("Front Leg", style='Title')
                    doc.add_paragraph("Front leg brace and stability are rooted from the pelvis clearing as a cleared pelvis "
                                      "restricts varus/valgus movement on the knee while enabling the front knee to extend "
                                      "through ball release."
                                      "", style='Heading 2')
                elif column_name == "Lead_Leg_Midpoint":
                    doc.add_paragraph("Ground Reaction Forces", style='Title')
                    doc.add_paragraph("Ground reaction forces are the measurement of the force the front leg puts into the ground "
                                      "throughout the pitching motion. Ground reaction forces have been linked to increase ball "
                                      "velocity as well as kinematic variables that are advantageous for both ball velocity and "
                                      "shoulder/arm health/longevity. The ideal ground reaction force has the peaks of the ground "
                                      "reaction force occurring midway between front leg footplant and ball release."
                                      "", style='Heading 2')
                elif column_name == "Horizontal_Abduction":
                    doc.add_paragraph("Horizontal Abduction at Footplant", style='Title')
                    doc.add_paragraph("Horizontal abduction is the measurement of how far behind the body the arm is at footplant. "
                                      "One of the biggest indicators of ball velocity as increased abdication allows the arm to "
                                      "efficiently trail the body, allowing more force created from the front leg to transmit up "
                                      "the body and act on the ball."
                                      "", style='Heading 2')
                elif column_name == "Shld_ER_Footplant":
                    doc.add_paragraph("Shoulder External Rotation", style='Title')
                    doc.add_paragraph("Shoulder external rotation at footplant defines an early, on-time, or late arm. Late arms "
                                      "are defined as below 33 degrees and an early arm is above 77 degrees at footplant. An arm "
                                      "that is on-time increases the arms ability to efficiently sync up with the body while "
                                      "reducing the risk of injury. "
                                      "Maximum shoulder external rotation is better known as “layback”, has been correlated to "
                                      "ball velocity and is an indicator of overall shoulder mobility."
                                      "", style='Heading 2')
                elif column_name == "Lateral_Trunk_Tilt":
                    doc.add_paragraph("Lateral Trunk Tilt at Ball Release", style='Title')
                    doc.add_paragraph("Lateral trunk tilt is how tilted one is at ball release."
                                      "", style='Heading 2')
                elif column_name == "Pelvis_Ang_Velo":
                    doc.add_paragraph("Kinematic Sequence", style='Title')
                    doc.add_paragraph("Kinematic sequencing is the order and speed at which the pelvis, torso, and arm reach "
                                      "their peak rotational velocity. The order should be in sequential order of pelvis, "
                                      "followed by torso, then arm. The higher the maximum rotational velocity for all segments, "
                                      "the better."
                                      "", style='Heading 2')
                elif column_name == "MPH":
                    doc.add_paragraph("MPH", style='Title')
                
                # Dictionary to specify the width for each variable
                widths = {
                    'Score': 6,
                    'Linear_Pelvis_Speed': 4.6,
                    'HSS_Footplant': 6,
                    'Pelvis_Ang_Footplant': 4.6,
                    'Trunk_Ang_Footplant': 4.6,
                    'Pelvic_Obl': 4.6,
                    'Front_Leg_Brace': 4.4,
                    'Front_Leg_Var_Val': 4.4,
                    'Lead_Leg_Midpoint': 6,
                    'Horizontal_Abduction': 6,
                    'Shld_ER_Footplant': 4,
                    'Shld_ER_Max': 4,
                    'Lateral_Trunk_Tilt': 5,
                    'Pelvis_Ang_Velo': 4.6,
                    'Torso_Ang_Velo': 4.6,
                    'Arm_Ang_Velo': 4.6,
                    'MPH': 6
                }
                
                # Add the histogram image to the document
                if column_name in widths:
                    paragraph = doc.add_paragraph()
                    run = paragraph.add_run()
                    run.add_picture(histogram_filename, width=Inches(widths[column_name]))
                    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                else:
                    paragraph = doc.add_paragraph()
                    run = paragraph.add_run()
                    run.add_picture(histogram_filename, width=Inches(4))  # Default width
                    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

                if column_name in variables_with_page_breaks and i < len(variables_to_include) - 1:
                    doc.add_page_break()

except sqlite3.Error as e:
    print(f"SQLite error: {e}")

finally:
    conn_new.close()
    conn_reference.close()

# Save the document
doc.save(docx_filename)


# Save the document
doc.save(docx_filename)


# Move files
source_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"
kinematic_data_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\Kinematic Data\High School"
mph_data_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\MPH Data\High School"

for file_name in os.listdir(source_dir):
    source_file_path = os.path.join(source_dir, file_name)
    if 'data' in file_name.lower():
        destination_file_path = os.path.join(kinematic_data_dir, file_name)
    else:
        destination_file_path = os.path.join(mph_data_dir, file_name)
    shutil.move(source_file_path, destination_file_path)
    print(f"File '{file_name}' moved to: {destination_file_path}")

print("Files moved successfully!")

File 'session.xml' copied and renamed to: H:/Pitching/Data/Castillo, JC\temp_rename\session_JC Castillo_12-09-24.xml
File 'session_data.xml' copied and renamed to: H:/Pitching/Data/Castillo, JC\temp_rename\session_data_JC Castillo_12-09-24.xml
File 'session_JC Castillo_12-09-24.xml' copied to: G:\My Drive\Pitching Data\New Data\session_JC Castillo_12-09-24.xml
File 'session_data_JC Castillo_12-09-24.xml' copied to: G:\My Drive\Pitching Data\New Data\session_data_JC Castillo_12-09-24.xml
File 'session_JC Castillo_12-09-24.xml' moved to: C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data\session_JC Castillo_12-09-24.xml
File 'session_data_JC Castillo_12-09-24.xml' moved to: C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data\session_data_JC Castillo_12-09-24.xml
XML files copied successfully!
1.1406520161123126 2.252998078132626 0.39133919735184963 2.110326277826986
1.1649913398567917 2.362557007821855 0.36194842481312123 2.1546456285385567
1.1268287589